# todo (18/05/2021)
* __rsync data of one participant (locEMsexp)__
* __basic visualization__
* __experimental design__
* __try first level_glm__

# todo next
* try concatened runs instead of average
* save the different glmn output as nifti images and look at them in sofwares
* put them in pycortex flatmaps and draw areas
% GLM of pRF to later delete
% GLM of smooth pursuit
% comparison smooth pursuit vs saccade
% other subject
% mni average subjects

# questions
* is it better to do glm from average runs or concatenated runs ?

In [1]:
# imports
import os
import numpy as np
import pandas as pd
from nilearn import image, datasets, plotting, surface
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
from nilearn.plotting import plot_design_matrix, plot_stat_map, plot_anat, plot_img
import matplotlib.pyplot as plt

/home/vmorita/anaconda3/envs/idp/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
# inputs
# user_login = 'mszinte'
user_login = 'vmorita'
subject = 'sub-01'
session = 'ses-01'
task = 'SacLoc'
space = 'T1w'
run = 'run-1'
preproc = 'fmriprep_dct'
tr_dur = 1.2
cwd = os.getcwd()

In [4]:
cwd = os.getcwd()
cwd

'/home/vmorita/projects/PredictEye/mri_analysis/post_fit'

In [ ]:
# rsync func data
trans_cmd = 'rsync -avuz --progress'
orig_file = '{}@login.mesocentre.univ-amu.fr:/scratch/mszinte/data/PredictEye/pp_data/{}/func'.\
                format(user_login,subject)
dest_file = '{}/pp_data/{}/func'.format(cwd,subject)
try: os.makedirs(dest_file)
except: pass

print("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))
# os.system("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))

In [ ]:
# rsync anat data
trans_cmd = 'rsync -avuz --progress'
orig_file = '{user_login}@login.mesocentre.univ-amu.fr:/scratch/mszinte/data/PredictEye/deriv_data/fmriprep/fmriprep/{subject}/anat/{subject}_desc-preproc_T1w.nii.gz'.\
                format(user_login=user_login, subject=subject)
dest_file = '{}/pp_data/{}/anat'.format(cwd,subject)
try: os.makedirs(dest_file)
except: pass

print("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))
# os.system("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))

In [ ]:
# rsync events data
trans_cmd = 'rsync -avuz --progress'
orig_file = '{user_login}@login.mesocentre.univ-amu.fr:/scratch/mszinte/data/PredictEye/bids_data/{subject}/{session}/func/{subject}_{session}_task-{task}_run-*_events.tsv'.\
                format(user_login=user_login, subject=subject, session=session, task=task)
dest_file = '{}/pp_data/{}/events'.format(cwd,subject)
try: os.makedirs(dest_file)
except: pass

print("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))
# os.system("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))

In [ ]:
# rsync surf data
trans_cmd = 'rsync -avuz --progress'
orig_file = '{user_login}@login.mesocentre.univ-amu.fr:/scratch/mszinte/data/PredictEye/deriv_data/fmriprep/freesurfer/{subject}/surf/'.\
                format(user_login=user_login, subject=subject)
dest_file = '{}/pp_data/{}/surf'.format(cwd,subject)
try: os.makedirs(dest_file)
except: pass

print("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))
# os.system("{cmd} {orig} {dest}".format(cmd = trans_cmd, orig = orig_file, dest = dest_file))

In [ ]:
# plot t1w/t2w subject anat
anat_img = "pp_data/{subject}/anat/{subject}_desc-preproc_T1w.nii.gz".\
                format(subject=subject)

plotting.plot_anat(anat_img = anat_img)

In [3]:
# plot epi tr = 1
file_img     = "pp_data/{subject}/func/fmriprep_dct/{subject}_task-{task}_space-{space}_{run}_{preproc}.nii.gz".\
                format(subject=subject,task=task,space=space,run=run,preproc=preproc)
file_img_avg = "pp_data/{subject}/func/{subject}_task-{task}_space-{space}_{preproc}_avg.nii.gz".\
                format(subject=subject,task=task,space=space,preproc=preproc)
plotting.plot_epi(epi_img = image.index_img(file_img, 0), cmap='viridis', black_bg=False)

ValueError: File not found: 'pp_data/sub-01/func/fmriprep_dct/sub-01_task-SacLoc_space-T1w_run-1_fmriprep_dct.nii.gz'

In [ ]:
# create design table
design_file_run1 = '{cwd}/pp_data/{subject}/events/{subject}_{session}_task-{task}_{run}_events.tsv'.\
                        format(cwd=cwd, subject=subject, session=session, task=task, run='run-01')
events_run1 = pd.read_table(design_file_run1)

design_file_run2 = '{cwd}/pp_data/{subject}/events/{subject}_{session}_task-{task}_{run}_events.tsv'.\
                        format(cwd=cwd, subject=subject, session=session, task=task, run='run-02')
events_run2 = pd.read_table(design_file_run2)

In [ ]:
events_run1_glm = events_run1[['onset','duration','trial_type']].copy(deep=True)
events_run1_glm.replace({'trial_type': {3: 'Fix', 1: 'Sac', 2: 'Pur'}},inplace=True)
events_run1_glm_groups = events_run1_glm.groupby((events_run1_glm.trial_type!=events_run1_glm.trial_type.shift()).cumsum())

events_run1_glm = pd.DataFrame([], columns=['onset', 'duration', 'trial_type'])
for idx, group in enumerate(events_run1_glm_groups):
    onset = group[1]['onset'][group[1].index[0]]
    dur = np.round(sum(group[1]['duration']*10))/10
    type = group[1]['trial_type'][group[1].index[0]]

    events_run1_glm = events_run1_glm.append(pd.Series([onset,dur,type],index =['onset', 'duration', 'trial_type']), ignore_index=True)

events_run1_glm['onset'] = np.floor((events_run1_glm['onset']-events_run1_glm['onset'][0])*10)/10
events_run1_glm

In [ ]:
onset = [    0*tr_dur,
             16*tr_dur,
             (16+32)*tr_dur,
             (16+32+16)*tr_dur,
             (16+32+16+32)*tr_dur,
             (16+32+16+32+16)*tr_dur,
             (16+32+16+32+16+32)*tr_dur,
             (16+32+16+32+16+32+16)*tr_dur,
             (16+32+16+32+16+32+16+32)*tr_dur]

duration = [ 16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             32*tr_dur,
             16*tr_dur,
             ]

trial_type = ['Fix','Sac','Fix','Sac','Fix','Sac','Fix','Sac','Fix']

data = {'onset':onset,
        'duration':duration,
        'trial_type':trial_type}

events_run1_glm = pd.DataFrame(data)
events_run1_glm

In [ ]:
# first level GLM
fmri_glm = FirstLevelModel(t_r=tr_dur,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model='cosine',
                           mask_img = False,
                           high_pass=.01)

fmri_img = file_img
fmri_glm = fmri_glm.fit(fmri_img, events_run1_glm)

In [ ]:
design_matrix = fmri_glm.design_matrices_[0]
plot_design_matrix(design_matrix)
plt.show()

In [ ]:
plt.plot(design_matrix['Sac'])
plt.xlabel('Time (seconds)')
plt.title('Saccade periods')
plt.show()

In [ ]:
plt.plot(design_matrix['Fix'])
plt.xlabel('Time (seconds)')
plt.title('Fixation periods')
plt.show()

In [ ]:
conditions = { 'Fix': np.array([1., 0., 0., 0., 0., 0., 0.]),
               'Sac': np.array([0., 1., 0., 0., 0., 0., 0.])}

sac_minus_fix = conditions['Sac'] - conditions['Fix']
plotting.plot_contrast_matrix(sac_minus_fix, design_matrix=design_matrix)

In [ ]:
eff_map = fmri_glm.compute_contrast(sac_minus_fix,
                                    output_type='effect_size')

In [ ]:
z_map = fmri_glm.compute_contrast(sac_minus_fix,
                                  output_type='z_score')

In [ ]:
# save z_map TO-DO
file_zmap     = "pp_data/{subject}/func/fmriprep_dct/{subject}_task-{task}_space-{space}_{run}_{preproc}_zmap.nii.gz".\
                format(subject=subject,task=task,space=space,run=run,preproc=preproc)
file_zmap_avg = "pp_data/{subject}/func/{subject}_task-{task}_space-{space}_{preproc}_avg_zmap.nii.gz".\
                format(subject=subject,task=task,space=space,preproc=preproc)


In [ ]:
# plot_stat_map(z_map, bg_img=anat_img, threshold=3.0,
#               display_mode='z', cut_coords=3, black_bg=True,
#               title='Active minus Rest (Z>3)')
# plt.show()

# interactive plot
plotting.view_img(z_map, bg_img=anat_img, threshold=3, vmax=12, cut_coords=[-42, -16, 52], title="Active minus Rest (Z>3)") 

In [ ]:
# https://nilearn.github.io/auto_examples/01_plotting/plot_3d_map_to_surface_projection.html#sphx-glr-auto-examples-01-plotting-plot-3d-map-to-surface-projection-py
surf = {
    'pial_left': '{}/pp_data/{}/surf/lh.pial'.format(cwd,subject),
    'pial_right': '{}/pp_data/{}/surf/rh.pial'.format(cwd,subject),
    'infl_left': '{}/pp_data/{}/surf/lh.inflated'.format(cwd,subject),
    'infl_right': '{}/pp_data/{}/surf/rh.inflated'.format(cwd,subject),
    'sulc_left': '{}/pp_data/{}/surf/lh.sulc'.format(cwd,subject),
    'sulc_right': '{}/pp_data/{}/surf/rh.sulc'.format(cwd,subject)
}

texture = surface.vol_to_surf(z_map, surf['pial_right'])
plotting.plot_surf_stat_map(surf['infl_right'], texture, hemi='right',
                            title='Surface right hemisphere', colorbar=True,
                            threshold=1., bg_map=surf['sulc_right'])

# plotting.view_img_on_surf(z_map, surf_mesh=surf, threshold='90%')

In [ ]:
_, threshold = threshold_stats_img(z_map, alpha=.001, height_control='fpr')
print('Uncorrected p<0.001 threshold: %.3f' % threshold)
# plot_stat_map(z_map, bg_img=anat_img, threshold=threshold,
#               display_mode='z', cut_coords=3, black_bg=True,
#               title='Active minus Rest (p<0.001)')
# plt.show()

# interactive plot
plotting.view_img(z_map, bg_img=anat_img, threshold=threshold, vmax=12, cut_coords=[-42, -16, 52], title="Active minus Rest (p<0.001)") 


In [ ]:
img_map, threshold = threshold_stats_img(
    z_map, alpha=.05, height_control='bonferroni')
print('Bonferroni-corrected, p<0.05 threshold: %.3f' % threshold)
# plot_stat_map(z_map, bg_img=anat_img, threshold=threshold,
#               display_mode='z', cut_coords=3, black_bg=True,
#               title='Active minus Rest (p<0.05, corrected)')
# plt.show()

plotting.view_img(z_map, bg_img=anat_img, threshold=threshold, vmax=12, cut_coords=[-42, -16, 52], title="Active minus Rest (p<0.05, corrected)") 


In [ ]:
_, threshold = threshold_stats_img(z_map, alpha=.05, height_control='fdr')
print('False Discovery rate = 0.05 threshold: %.3f' % threshold)
plot_stat_map(z_map, bg_img=anat_img, threshold=threshold,
              display_mode='z', cut_coords=3, black_bg=True,
              title='Active minus Rest (fdr=0.05)')
plt.show()

In [ ]:
clean_map, threshold = threshold_stats_img(
    z_map, alpha=.05, height_control='fdr', cluster_threshold=10)
plot_stat_map(clean_map, bg_img=anat_img, threshold=threshold,
              display_mode='z', cut_coords=3, black_bg=True,
              title='Active minus Rest (fdr=0.05), clusters > 10 voxels')
plt.show()